In [8]:
import pandas as pd; import numpy as np;
import os; import math; import copy
#import torch; import torch.nn as nn
#from transformers import AutoTokenizer

In [12]:
def get_batch(text, num_scripts=4, batch_size=64, block_size=256):
    data = list(text.sample(num_scripts)['content'])
    data = tokenizer(data, truncation=True)['input_ids']
    data = [add_section_tokens(sent) for sent in data if len(sent)>block_size]
    X, Y = format_one_script(data[0], batch_size=batch_size, block_size=block_size)
    for d in data[1:]:
        x, y = format_one_script(d, batch_size=batch_size, block_size=block_size)
        X, Y = torch.concat([X, x], dim=0), torch.concat([Y, y], dim=0)
    del data
    return X.to(device), Y.to(device)

def add_section_tokens(sent, prob=0.5):
    sent_break_length = len(sent)//3
    pre = pre_s + sent[:sent_break_length] + pre_e
    mid = sent[sent_break_length:len(sent)-sent_break_length]
    mid = mid_s + mid + mid_e
    suf = suf_s + sent[-sent_break_length:] + suf_e
    return pre + mid + suf if np.random.random(1)[0] >= prob else pre + suf + mid

def format_one_script(data, batch_size=64, block_size=256):
    data = torch.tensor(data, dtype=torch.long)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    del data
    return x, y

#data
#path = '/Users/kchen/Documents/Machine Learning/Models/Transformer Architecture/CodeLLM/Code/data/the_stack/'
#text = pd.read_parquet(path+'data-00000-of-00144.parquet')
#text = text[text['size']>1000]

token indices sequence length is longer than the specified maximum sequence length

This means you're encoding a sequence that is larger than the max sequence the model can handle (which is 512 tokens). This is not an error but a warning; if you pass that sequence to the model it will crash as it cannot handle such a long sequence.

You can truncate the sequence: seq = seq[:512] or use the max_length tokenizer parameter so that it handles it on its own.